In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 16 11:39:49 2019

#### Wifi Challenge
# Predict location

#@author: Aline Barbosa Alves
"""

'\nCreated on Wed Oct 16 11:39:49 2019\n\n#### Wifi Challenge\n# Predict location\n\n#@author: Aline Barbosa Alves\n'

In [23]:
# Import packages

import pandas as pd
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score, \
                            classification_report, confusion_matrix, \
                            cohen_kappa_score
                            
from sklearn.model_selection import cross_val_score
import plotly.express as px
import numpy as np
from plotly.offline import plot
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from math import sqrt, pi

#### Ignacio: Added some usefull modules.
import glob, os
from pathlib import Path

In [34]:
"""Import Data"""

# Save filepath to variable for easier access

####Ignacio: Aline, I commented out those lines because you provide the absolute path which is dependent on your machine.
#### If I run it on my machine, they will not work as the folder will not exist. Therefore, you need to use relative paths.

#training_data_path = '/home/aline/Documentos/Ubiqum/IoT Analytics/WifiLocationing/data/trainingData.csv'
#validation_data_path = '/home/aline/Documentos/Ubiqum/IoT Analytics/WifiLocationing/data/validationData.csv'
#test_data_path = '/home/aline/Documentos/Ubiqum/IoT Analytics/WifiLocationing/data/testData.csv'

print(os.getcwd())

#### Ignacio: Used relative paths.
training_data_path = '../data/trainingData.csv'
validation_data_path = '../data/validationData.csv'
test_data_path = '../data/testData.csv'


C:\Users\Ignacio\Documents\UBIQUM\MENTOR\DA119\Aline\Wifi-Locationing-1\script


In [35]:
# read the data and store data in DataFrame

training_data = pd.read_csv(training_data_path) 
validation_data = pd.read_csv(validation_data_path)
test_data = pd.read_csv(test_data_path)

In [38]:
"""Get to know the data"""
# Print a summary of the data

#### Ignacio: Added some test to third parties about what you are showing.
#### In addition, we use the "display" function to show t
#training_data.describe()
#validation_data.describe()

print("Summary of Training dataset")
display(training_data.describe())
print()
print("Summary of Validation dataset")
display(validation_data.describe())

Summary of Training dataset


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
count,19937.000000,19937.000000,19937.0,19937.0,19937.000000,19937.000000,19937.000000,19937.000000,19937.000000,19937.000000,...,19937.0,19937.000000,1.993700e+04,19937.000000,19937.000000,19937.000000,19937.000000,19937.000000,19937.000000,1.993700e+04
mean,99.823644,99.820936,100.0,100.0,99.613733,97.130461,94.733661,93.820234,94.693936,99.163766,...,100.0,-7464.275947,4.864871e+06,1.674575,1.212820,148.429954,1.833024,9.068014,13.021869,1.371421e+09
std,5.866842,5.798156,0.0,0.0,8.615657,22.931890,30.541335,33.010404,30.305084,12.634045,...,0.0,123.402010,6.693318e+01,1.223078,0.833139,58.342106,0.372964,4.988720,5.362410,5.572054e+05
min,-97.000000,-90.000000,100.0,100.0,-97.000000,-98.000000,-99.000000,-98.000000,-98.000000,-99.000000,...,100.0,-7691.338400,4.864746e+06,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.369909e+09
25%,100.000000,100.000000,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,-7594.737000,4.864821e+06,1.000000,0.000000,110.000000,2.000000,5.000000,8.000000,1.371056e+09
50%,100.000000,100.000000,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,-7423.060900,4.864852e+06,2.000000,1.000000,129.000000,2.000000,11.000000,13.000000,1.371716e+09
75%,100.000000,100.000000,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,-7359.193000,4.864930e+06,3.000000,2.000000,207.000000,2.000000,13.000000,14.000000,1.371721e+09
max,100.000000,100.000000,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.0,-7300.818990,4.865017e+06,4.000000,2.000000,254.000000,2.000000,18.000000,24.000000,1.371738e+09



Summary of Validation dataset


,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
count,1111.000000,1111.0,1111.000000,1111.000000,1111.0,1111.0,1111.0,1111.000000,1111.000000,1111.000000,...,1111.000000,1111.000000,1.111000e+03,1111.000000,1111.000000,1111.0,1111.0,1111.0,1111.000000,1.111000e+03
mean,98.627363,100.0,99.665167,99.665167,100.0,100.0,100.0,94.297030,90.655266,98.811881,...,99.843384,-7529.197448,4.864902e+06,1.571557,0.758776,0.0,0.0,0.0,11.923492,1.380604e+09
std,16.127245,0.0,7.888265,7.888607,0.0,0.0,0.0,32.148809,39.886991,14.931667,...,5.220261,120.209336,7.027280e+01,1.001828,0.816139,0.0,0.0,0.0,6.556236,5.003222e+05
min,-94.000000,100.0,-87.000000,-88.000000,100.0,100.0,100.0,-99.000000,-96.000000,-98.000000,...,-74.000000,-7695.938755,4.864748e+06,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.379578e+09
25%,100.000000,100.0,100.000000,100.000000,100.0,100.0,100.0,100.000000,100.000000,100.000000,...,100.000000,-7637.423800,4.864843e+06,1.000000,0.000000,0.0,0.0,0.0,9.000000,1.380189e+09
50%,100.000000,100.0,100.000000,100.000000,100.0,100.0,100.0,100.000000,100.000000,100.000000,...,100.000000,-7560.376300,4.864915e+06,1.000000,1.000000,0.0,0.0,0.0,13.000000,1.380873e+09
75%,100.000000,100.0,100.000000,100.000000,100.0,100.0,100.0,100.000000,100.000000,100.000000,...,100.000000,-7420.539659,4.864967e+06,2.000000,1.000000,0.0,0.0,0.0,15.000000,1.380875e+09
max,100.000000,100.0,100.000000,100.000000,100.0,100.0,100.0,100.000000,100.000000,100.000000,...,100.000000,-7299.786517,4.865017e+06,4.000000,2.000000,0.0,0.0,0.0,21.000000,1.381248e+09


In [42]:
# Columns

#training_data.columns
#validation_data.columns

#### Ignacio: Again the same thing as before and this time making it nicer
print("Column names of the Training dataset")
print()
print(training_data.columns.tolist())
print()
print("Column names of the Validation dataset")
print()
print(validation_data.columns.tolist())

Column names of the Training dataset

['WAP001', 'WAP002', 'WAP003', 'WAP004', 'WAP005', 'WAP006', 'WAP007', 'WAP008', 'WAP009', 'WAP010', 'WAP011', 'WAP012', 'WAP013', 'WAP014', 'WAP015', 'WAP016', 'WAP017', 'WAP018', 'WAP019', 'WAP020', 'WAP021', 'WAP022', 'WAP023', 'WAP024', 'WAP025', 'WAP026', 'WAP027', 'WAP028', 'WAP029', 'WAP030', 'WAP031', 'WAP032', 'WAP033', 'WAP034', 'WAP035', 'WAP036', 'WAP037', 'WAP038', 'WAP039', 'WAP040', 'WAP041', 'WAP042', 'WAP043', 'WAP044', 'WAP045', 'WAP046', 'WAP047', 'WAP048', 'WAP049', 'WAP050', 'WAP051', 'WAP052', 'WAP053', 'WAP054', 'WAP055', 'WAP056', 'WAP057', 'WAP058', 'WAP059', 'WAP060', 'WAP061', 'WAP062', 'WAP063', 'WAP064', 'WAP065', 'WAP066', 'WAP067', 'WAP068', 'WAP069', 'WAP070', 'WAP071', 'WAP072', 'WAP073', 'WAP074', 'WAP075', 'WAP076', 'WAP077', 'WAP078', 'WAP079', 'WAP080', 'WAP081', 'WAP082', 'WAP083', 'WAP084', 'WAP085', 'WAP086', 'WAP087', 'WAP088', 'WAP089', 'WAP090', 'WAP091', 'WAP092', 'WAP093', 'WAP094', 'WAP095', 'WAP096', '

In [44]:
# Missing data

#training_data.isnull().any().sum()
#validation_data.isnull().any().sum()

#### Ignacio: Counting how many missing values has each dataset
print("Number on missing values in the Training dataset: ",training_data.isnull().any().sum())
print("Number on missing values in the Validation dataset: ",validation_data.isnull().any().sum())

Number on missing values in the Training dataset:  0
Number on missing values in the Validation dataset:  0


In [45]:
"""Pre processing"""

# Feature selection - Pick only WAPs

#### Ignacio: Alinne, you should explain why do you merge both datasets.
#### If you merge the training and validation sets into a new dataset called
#### training, then how do you will call the new training set in the split of
#### the training set?

training = training_data.iloc[:,0:520]
training = training.append(validation_data.iloc[:,0:520])
validation = test_data.iloc[:,0:520]


In [46]:
# Targets

latitude = training_data.LATITUDE.append(validation_data.LATITUDE)
longitude = training_data.LONGITUDE.append(validation_data.LONGITUDE)
floor = training_data.FLOOR.append(validation_data.FLOOR)
building = training_data.BUILDINGID.append(validation_data.BUILDINGID)

In [49]:
# Set floor and building as categorical data

floor = pd.Series(floor, dtype="category")
building = pd.Series(building, dtype="category")

In [50]:
# Cross-validation

def get_score_rfr(n_estimators):
    my_pipeline = Pipeline(steps=[
        ('model', RandomForestRegressor(n_estimators=n_estimators, 
                                        random_state=0))
    ])
    scores = -1 * cross_val_score(my_pipeline, training, latitude,
                                      cv=3,
                                      scoring='neg_mean_absolute_error')
    return scores.mean()

In [52]:
number_of_trees = [50, 100, 150, 200, 250, 300, 350, 400] # 200

results = {}

for i in number_of_trees:
    results.update({i:get_score_rfr(i)})

In [ ]:
"""Normalize"""

#### Ignacio: Why do you want to normalize by range if all the waps are in the same scale?
#### According to the current scale, a signal value of "0" means a very strong signal,
#### while weak signals have a big negative value. Therefore, the weaker the signal, the 
#### more negative value. 
#### To confuse things, locations were a no signal has been recorded are given a value of "100".
#### If you replace those values with "0", you are giving them a high "signal" which is not 
#### what you want. In this sense, replacing the "100" by a value smaller than the minimum singal
#### in the dataset will have more sense ( of course, the actual value can influence a bit your
#### model predictions ).

#### Of course, replacing the 100's values by zero has a lot of sense because maybe you're 
#### going to use methods based on distance and "no signal" can't be assigned a big value 
#### like 100. Otherwise, weak signals contribute a lot to the distance. 
#### But then, the "old zero" signals will have to be shifted to a big value to have a big
#### contribution to the signal. Moreover, you have to make this shift carefully before sending 100's to 0.
#### In simple words, you want that the good signals have a big contribution and conversely
#### weak signals to have low contribution. 

# Create function to transform putting 'not found' signals (100) as 
# the min and normalize data

def normalize(df):
    for i in range(0,len(df.columns)):
        # If its only 'not found', put zero
        if (df.iloc[:,[i]].min()==df.iloc[:,[i]].max()).bool():
            df.iloc[:,[i]] = 0
        else:
            # Replace 'not found' with the min
            df.iloc[:,[i]] = \
            df.iloc[:,[i]].replace(100, df.iloc[:,[i]].min())

            # Normalize data            
            if (df.iloc[:,[i]].min()==df.iloc[:,[i]].max()).bool():
                df.iloc[:,[i]] = 0
            else:
                df.iloc[:,[i]] = \
                (df.iloc[:,[i]] - df.iloc[:,[i]].min())/ \
                (df.iloc[:,[i]].max() - df.iloc[:,[i]].min())
    return df
